In [1]:
# Change directory to the root of the project
import os 
os.chdir('..')
os.chdir('..')
os.chdir('..')
print(f"Working directory: {os.getcwd()}")

Working directory: /Users/eohjelle/Documents/2025-dots-and-boxes/dots-and-boxes


In [2]:
from applications.tic_tac_toe.models.linear_attention_transformer import LinearAttentionTransformerInitParams
import torch

# Initialize parameters

## Model parameters
model_type = 'linear_attention_transformer'
model_params: LinearAttentionTransformerInitParams = {
    'embed_dim': 32,
    'ff_dim': 128,
    'num_heads': 4,
}
device = torch.device('mps')
model_name = 'tic_tac_toe_linear_attention_transformer'

## Initialize new model
load_model = None
load_model_params = {}


## Optimizer parameters
optimizer_type = 'adam'
optimizer_params = {
    'lr': 1e-2,
    'betas': (0.9, 0.999),
    'eps': 1e-8,
    'weight_decay': 1e-4,
    'amsgrad': False
}

## Learning scheduler parameters
lr_scheduler_type = 'plateau'
lr_scheduler_params = {
    'factor': 0.5,
    'patience': 25,
    'cooldown': 50,
    'min_lr': 1e-6
}

## Training parameters
training_method = 'supervised'
trainer_params = {}
training_params = {
    'epochs': 1000,
    'batch_size': 256,
    'eval_freq': 25,
    'checkpoint_freq': 50,
    'mask_illegal_moves': False,
    'mask_value': -20.0, # Doesn't matter when mask_illegal_moves is False
    'checkpoint_dir': 'checkpoints',
    'start_at': 1
}

## Load training data from wandb
load_replay_buffer = 'from_wandb'
load_replay_buffer_params = {
    'project': 'AlphaZero-TicTacToe',
    'artifact_name': f'tic_tac_toe_TokenizedTensorMapping_training_data',
    'artifact_version': 'latest'
}


In [3]:
# Initialize wandb run
import wandb

run_name = 'Linear Attention Transformer 9'
notes = 'Same as v1 I think, seeing if the good run can be recreated.'

config = {
    'model_type': model_type,
    'model_params': model_params,
    'optimizer_type': optimizer_type,
    'optimizer_params': optimizer_params,
    'lr_scheduler_type': lr_scheduler_type,
    'lr_scheduler_params': lr_scheduler_params,
    'training_method': training_method,
    'trainer_params': trainer_params,
    'training_params': training_params
}

run = wandb.init(
    project='AlphaZero-TicTacToe',
    name=run_name,
    config=config,
    notes=notes
)

wandb: Currently logged in as: eohjelle (eigenway) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [4]:
# Perform training

from applications.tic_tac_toe.train import train

model_interface = train(
    model_type=model_type,
    model_params=model_params,
    device=device,
    model_name=model_name,
    optimizer_type=optimizer_type,
    optimizer_params=optimizer_params,
    lr_scheduler_type=lr_scheduler_type,
    lr_scheduler_params=lr_scheduler_params,
    training_method=training_method,
    trainer_params=trainer_params,
    training_params=training_params,
    load_model=load_model,
    load_model_params=load_model_params,
    load_replay_buffer=load_replay_buffer,
    load_replay_buffer_params=load_replay_buffer_params,
    wandb_run=run
)

wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
wandb:   1 of 1 files downloaded.  
/Users/eohjelle/Documents/2025-dots-and-boxes/dots-and-boxes/core/data_structures.py:94: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue

Epoch 1/1000, Train Loss: 1.8951, Val Loss: 1.6927, Learning rate: 0.010000
Epoch 2/1000, Train Loss: 1.6972, Val Loss: 1.6302, Learning rate: 0.010000
Epoch 3/1000, Train Loss: 1.6093, Val Loss: 1.6223, Learning rate: 0.010000
Epoch 4/1000, Train Loss: 1.5962, Val Loss: 1.5970, Learning rate: 0.010000
Epoch 5/1000, Train Loss: 1.5488, Val Loss: 1.5901, Learning rate: 0.010000
Epoch 6/1000, Train Loss: 1.5593, Val Loss: 1.6007, Learning rate: 0.010000
Epoch 7/1000, Train Loss: 1.5512, Val Loss: 1.6038, Learning rate: 0.010000
Epoch 8/1000, Train Loss: 1.5554, Val Loss: 1.6057, Learning rate: 0.010000
Epoch 9/1000, Train Loss: 1.5481, Val Loss: 1.6095, Learning rate: 0.010000
Epoch 10/1000, Train Loss: 1.5470, Val Loss: 1.5845, Learning rate: 0.010000
Epoch 11/1000, Train Loss: 1.5411, Val Loss: 1.5825, Learning rate: 0.010000
Epoch 12/1000, Train Loss: 1.5304, Val Loss: 1.5898, Learning rate: 0.010000
Epoch 13/1000, Train Loss: 1.5386, Val Loss: 1.5911, Learning rate: 0.010000
Epoch 14

In [5]:
run.finish()

MCTS_draw_rate,▁▂▂▂▃▅▅▃▃▃▆▇▆▆▇▆▆▇█▇▇▇▇▇█▇▇▆▇▇▇▇▇▇█▇▇▇▇▇
MCTS_loss_rate,█▇▇▇▆▅▄▆▆▅▄▂▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
MCTS_score,▁▂▂▂▃▄▅▂▃▄▅▆▆█▇███▇▇█▇▇▇▇▇▇██▇█▇██▇▇▇██▇
MCTS_win_rate,▁▁▁▂▂▂▃▁▂▃▂▄▄▇▅█▇▅▅▆▇▅▅▅▄▄▅▇▆▆▆▆▆▇▄▆▅▆▆▅
Minimax_draw_rate,▁▂▃▂▂▃▅▂▂▄▄█▇███▇███████████████████████
Minimax_loss_rate,█▇▆▇▇▆▄▇▇▅▅▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Minimax_score,▁▂▃▂▂▃▅▂▂▄▄█▇███▇███████████████████████
Minimax_win_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
RandomAgent_draw_rate,▄▁▂▁▃▂▂▂▁▂▃▄▄▃▄▅▃▆▃▅▅▅▅▃▂█▅▂▃▅▅▄▅▄▄▇▇▆▄▄
RandomAgent_loss_rate,█▅▄▄▅▃▃▅▃▃▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▂▁▁▁▁
RandomAgent_score,▁▅▅▅▅▆▆▄▇▆█▇▇▇▇▇█▇▇▇▇▇▇██▆▇██▇▇▇▇▇▇▆▇▇▇▇


In [6]:
def print_model_parameters(model):
    """
    Print the total number of parameters in a PyTorch model,
    with a breakdown of trainable vs non-trainable parameters.
    
    Args:
        model: PyTorch model
    """
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params
    
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params:.2%})")
    print(f"Non-trainable parameters: {non_trainable_params:,} ({non_trainable_params/total_params:.2%})")
    
    # Optional: Print parameters by layer
    print("\nParameters by layer:")
    for name, param in model.named_parameters():
        print(f"{name}: {param.numel():,} parameters")

# Example usage
print_model_parameters(model_interface.model)

Total parameters: 13,410
Trainable parameters: 13,410 (100.00%)
Non-trainable parameters: 0 (0.00%)

Parameters by layer:
pos_embedding: 288 parameters
input_embedding.weight: 96 parameters
transformer_block.attn.q_emb.weight: 1,024 parameters
transformer_block.attn.q_emb.bias: 32 parameters
transformer_block.attn.k_emb.weight: 1,024 parameters
transformer_block.attn.k_emb.bias: 32 parameters
transformer_block.attn.v_emb.weight: 1,024 parameters
transformer_block.attn.v_emb.bias: 32 parameters
transformer_block.ff.0.weight: 32 parameters
transformer_block.ff.0.bias: 32 parameters
transformer_block.ff.1.weight: 4,096 parameters
transformer_block.ff.1.bias: 128 parameters
transformer_block.ff.3.weight: 4,096 parameters
transformer_block.ff.3.bias: 32 parameters
transformer_block.out_emb.weight: 1,024 parameters
transformer_block.out_emb.bias: 32 parameters
norm.weight: 32 parameters
norm.bias: 32 parameters
policy_head.weight: 32 parameters
policy_head.bias: 1 parameters
value_head.weigh